### Классификация текстов

#### Цель работы

Применить методы машинного обучения для решения задач классификации текстов.

#### Задания для выполнения

1. Загрузите датасет 20 newsgroups;
2. Познакомьтесь с описанием и структурой датасета. Описание можно найти в [документации](https://scikit-learn.org/stable/datasets/index.html#real-world-datasets).
3. Выведите информацию о количественных параметрах датасета;
4. Выведите несколько точек датасета (сами текстовые фрагменты и значение целевой переменной);
5. Разделите эти данные на тестовую и обучающую выборки;
6. Постройте модель наивного байесовского для классификации текстов;
7. Оцените качество модели на тестовой выборке с помощью следующих метрик:
    1. достоверность предсказания (accuracy);
    2. точность (precision);
    3. полнота (recall);
8. Постройте кривую обучения - график зависимости тестовой и обучающей эффективности от размера обучающей выборки.
9. Сделайте вывод о применимости модели.

#### Методические указания

Датасет 20 newsgroups - это один из известных модельных наборов данных для обучения методам классификации текстов на естественных языках. Его можно получить с помощью стандартных средств sklearn:

```py
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')
```

Превращение текста в набор численных признаков - это сама по себе нетривиальная задача. Часто эту задачу называют векторизация текста. От применяемого метода векторизации очень сильно зависит эффективность методов машинного обучения (наверное, даже больше, чем от самих методов классификации).

В библиотеку sklearn встроены несколько простых методов векторизации текстов. Они собраны в модуле sklearn.feature_extraction.text. Более подробно о них можно почитать в [документации](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Их применение довольно прямолинейно. Сначала нужно создать векторизатор, а затем преобразовать с помощью него текст в набор векторов:

```py
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
```

Обратите внимание, что данный метод преобразования составляет словарь токенов текста. Поэтому если мы применим его на другой выборке у нас получится другой словарь. Чтобы преобразовать другую выборку на основе того же самого словаря нужно применить другой метод:

```py
X = vectorizer.transform(['Something completely new.'])
```

Это поведение похоже на обучение модели. Векторизатор обучается на обучающей выборке, а затем используется для преобразования тестовой.

После векторизации исходных данных использование методов машинного обучения ничем не отличается от работы с числовыми данными. 

#### Контрольные вопросы

1. Какие выводы мы можем сделать на основании метрик модели, построенной в данной лабораторной работе?
2. Как представляется текст в методах машинного обучения? Какие основные методы векторизации существуют?
3. Какие задачи существую в области обработки естественных текстов?
4. (*) Что такое текстовые вложения (эмбеддинги)? Как они строятся и зачем применяются?

#### Дополнительные задания

1. Постройте модели классификации для данной задачи на основе следующих методов:
    1. логистическая регрессия (LogisticRegression);
    2. метод опорных векторов с гауссовым ядром (SVC);
    3. метод опорных векторов с полиномиальным ядром (SVC);
    4. метод k ближайших соседей (KNeighborsClassifier);
    5. многослойный перцептрон (MLP);
    6. другие методы по желанию;
2. Проанализируйте метрики каждой модели и сделайте выводы об их эффективности и применимости. Сравните эффективность всех этих моделей и выберите лучшую;
3. Для каждой модели из п.3 постройте кривые обучения и диагностируйте недо-/переобучение модели. Попробуйте изменить параметр регуляризации для улучшения результатов модели.
4. Сделайте замеры времени обучения для каждой модели. Сделайте вывод о сравнительной производительности моделей.
5. (*) Используйте глубокую нейронную сеть для решения той же задачи. Сравните ее эффективность и производительность с классическими моделями.


#### Задания для выполнения

1. Загрузите датасет 20 newsgroups;


In [1]:
# https://krakensystems.co/blog/2018/text-classification

In [10]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')

print("Number of articles: " + str(len(news.data)))
print("Number of diffrent categories: " + str(len(news.target_names)))

news.target_names

Number of articles: 18846
Number of diffrent categories: 20


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

2. Познакомьтесь с описанием и структурой датасета. Описание можно найти в [документации](https://scikit-learn.org/stable/datasets/index.html#real-world-datasets).


In [3]:
print(news.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

3. Выведите информацию о количественных параметрах датасета;


In [4]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
print("Number of articles: " + str(len(news.data)))
print("Number of diffrent categories: " + str(len(news.target_names)))

Number of articles: 18846
Number of diffrent categories: 20


4. Выведите несколько точек датасета (сами текстовые фрагменты и значение целевой переменной);


In [6]:
print(news.data[1121])

From: et@teal.csn.org (Eric H. Taylor)
Subject: Re: Gravity waves, was: Predicting gravity wave quantization & Cosmic Noise
Summary: Dong ....  Dong ....  Do I hear the death-knell of relativity?
Keywords: space, curvature, nothing, tesla
Nntp-Posting-Host: teal.csn.org
Organization: 4-L Laboratories
Distribution: World
Expires: Wed, 28 Apr 1993 06:00:00 GMT
Lines: 30

In article <C4KvJF.4qo@well.sf.ca.us> metares@well.sf.ca.us (Tom Van Flandern) writes:
>crb7q@kelvin.seas.Virginia.EDU (Cameron Randale Bass) writes:
>> Bruce.Scott@launchpad.unc.edu (Bruce Scott) writes:
>>> "Existence" is undefined unless it is synonymous with "observable" in
>>> physics.
>> [crb] Dong ....  Dong ....  Dong ....  Do I hear the death-knell of
>> string theory?
>
>     I agree.  You can add "dark matter" and quarks and a lot of other
>unobservable, purely theoretical constructs in physics to that list,
>including the omni-present "black holes."
>
>     Will Bruce argue that their existence can be inferre

5. Разделите эти данные на тестовую и обучающую выборки;


In [7]:
from sklearn.model_selection import train_test_split

x_train_tmp, x_test_tmp, y_train, y_test = train_test_split(news.data, news.target, test_size=0.2, random_state=11)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train_tmp)
x_test = vectorizer.transform(x_test_tmp)

6. Постройте модель наивного байесовского для классификации текстов;


In [11]:
import time
from sklearn import metrics

def train(classifier, X, y):
    start = time.time()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

    classifier.fit(X_train, y_train)
    end = time.time()
    
    #score.loc[classifier] = get_metrics(X_train, y_train)#.append(str(end - start))
    print("Accuracy: " + str(classifier.score(X_test, y_test)) + ", Time duration: " + str(end - start))
    return classifier

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

trial1 = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', MultinomialNB())])

train(trial1, news.data, news.target)

Accuracy: 0.8538461538461538, Time duration: 10.130509614944458


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', MultinomialNB())])

In [13]:
# теперь можно посмотреть, как изменятся метрики, если игнорировать наиболее распространенные слова
from nltk.corpus import stopwords

trial2 = Pipeline([ ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),('classifier', MultinomialNB())])

train(trial2, news.data, news.target)

Accuracy: 0.8806366047745358, Time duration: 13.893541812896729


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('classifier', MultinomialNB())])

In [14]:
# до этого классификатор наивного байеса использовал стандартное значение альфа = 1, теперь можно его подобрать
for alpha in [5, 0.5, 0.05, 0.005, 0.0005]:
    trial3 = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),('classifier', MultinomialNB(alpha=alpha))])
    train(trial3, news.data, news.target)

Accuracy: 0.8448275862068966, Time duration: 10.33638334274292
Accuracy: 0.8909814323607427, Time duration: 10.194643020629883
Accuracy: 0.9122015915119364, Time duration: 10.367761373519897
Accuracy: 0.9175066312997348, Time duration: 7.782767295837402
Accuracy: 0.9169761273209549, Time duration: 11.785998106002808


как можно заметить, лучшее качество достигается при alpha = 0.005

7. Оцените качество модели на тестовой выборке с помощью следующих метрик:
    1. достоверность предсказания (accuracy);
    2. точность (precision);
    3. полнота (recall);


In [12]:
scores = pd.DataFrame(
    columns=['accuracy', 'precision', 'recall', 'time', 'f1_score'],
    index=[]
)
scores

,accuracy,precision,recall,time,f1_score


In [18]:
from sklearn import metrics

def get_metrics(y_test, y_pred):
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='macro')
    recall = metrics.recall_score(y_test, y_pred, average='macro')
    f1_score = metrics.f1_score(y_test, y_pred, average='macro')
    return pd.Series(
        (accuracy, precision, recall, f1_score),
        index=['accuracy', 'precision', 'recall', 'f1_score']
    )

In [27]:
import time
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler

def model_and_metrics(model, x_train, y_train, x_test, y_test):
    start = time.time() # точка отсчета времени
    
#     model = make_pipeline(StandardScaler(with_mean=False, with_std=False), model)
#     model.fit(x_train, y_train)  # apply scaling on training data
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)  
    end = time.time() - start # время работы программы
    returned = get_metrics(y_test, y_pred).append(pd.Series((end), index = ['time']))
    return returned

In [28]:
from sklearn.linear_model import LogisticRegression

scores.loc['LogisticRegression'] = model_and_metrics(LogisticRegression(), x_train, y_train, x_test, y_test)

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
scores

# ХУЙ ЗНАЕТ КАК ЭТО ЗАПУСТИТЬ

# У МЕНЯ НЕ ХВАТАЕТ ОПЕРАТИВКИ

In [17]:
from sklearn import metrics

def get_metrics(y_test, y_pred):
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='micro')
    recall = metrics.recall_score(y_test, y_pred, average='micro')
    return pd.array((accuracy, precision, recall))

In [18]:
# изменили train, теперь он записывает все метрики модели в талицу
import time
from sklearn import metrics

def train(classifier, X, y):
    start = time.time()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

    classifier.fit(X_train, y_train)
    end = time.time()
    
    result = classifier.predict(X_train)
    time_scores.loc[classifier[1]] = end - start
    scores.loc[classifier[1]] = get_metrics(y_train, result)

In [ ]:
# Проходимся по всем моделям

# выполняется очень долго, поэтому лучше по одной
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# модели для обучения
estimators = [
#     GaussianNB(),
    MultinomialNB(),
    LogisticRegression(),
    svm.SVC(kernel='rbf'),
    svm.SVC(kernel='poly'),
    KNeighborsClassifier(),
    MLPClassifier()
]

for model in estimators:
    trial = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', model)])
    train(trial, news.data, news.target)

                      accuracy  precision    recall
LogisticRegression()   0.96982    0.96982   0.96982
MultinomialNB()       0.927899   0.927899  0.927899
                      accuracy  precision    recall
LogisticRegression()   0.96982    0.96982   0.96982
MultinomialNB()       0.927899   0.927899  0.927899
LogisticRegression()   0.96982    0.96982   0.96982


In [19]:
from sklearn.linear_model import LogisticRegression

trial = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', LogisticRegression())])

train(trial, news.data, news.target)

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
from sklearn.neighbors import KNeighborsClassifier

trial = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', KNeighborsClassifier())])

train(trial, news.data, news.target)

In [ ]:
from sklearn.neural_network import MLPClassifier

trial = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', MLPClassifier())])

train(trial, news.data, news.target)

In [ ]:
from sklearn.naive_bayes import GaussianNB

trial = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', GaussianNB())])

train(trial, news.data, news.target)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

trial = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', MultinomialNB())])

train(trial, news.data, news.target)

In [ ]:
scores

8. Постройте кривую обучения - график зависимости тестовой и обучающей эффективности от размера обучающей выборки.


9. Сделайте вывод о применимости модели.

#### Дополнительные задания

1. Постройте модели классификации для данной задачи на основе следующих методов:
    1. логистическая регрессия (LogisticRegression);
    2. метод опорных векторов с гауссовым ядром (SVC);
    3. метод опорных векторов с полиномиальным ядром (SVC);
    4. метод k ближайших соседей (KNeighborsClassifier);
    5. многослойный перцептрон (MLP);
    6. другие методы по желанию;


2. Проанализируйте метрики каждой модели и сделайте выводы об их эффективности и применимости. Сравните эффективность всех этих моделей и выберите лучшую;


3. Для каждой модели из п.3 постройте кривые обучения и диагностируйте недо-/переобучение модели. Попробуйте изменить параметр регуляризации для улучшения результатов модели.


4. Сделайте замеры времени обучения для каждой модели. Сделайте вывод о сравнительной производительности моделей.


5. (*) Используйте глубокую нейронную сеть для решения той же задачи. Сравните ее эффективность и производительность с классическими моделями. Нейронка с больше чем одним скрытым слоем